In [1]:
import csv
import os,sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy.stats import gaussian_kde

In [2]:
#prefix 
prefix='HI22'

#real photometry first
real_file = "%s.phot" % prefix
real_df = pd.read_csv(real_file, delim_whitespace=True, header=None) #create data frame 
real_df = real_df.iloc[:,:37] #select till relevant end of data frame (quality flag of I)

#name columns
columns=['extension','chip','x','y','chi','snr','sharp','round','maj_ax','crowd','type',
           'cts_V','sky_v','nrm_ct_rt_V','nrm_ct_rt_err_V','inst_vega_V','mag_V','Verr','chi_V','snr_V',
           'sharp_V','round_V','crowd_V','flag_V','cts_I','sky_I','nrm_ct_rt_I','nrm_ct_rt_err_I', 
           'inst_vega_I','mag_I','Ierr','chi_I','snr_I','sharp_I','round_I','crowd_I','flag_I']

In [3]:
#assign columns and preview
real_df.columns=columns

#fix y column if not on first chip 
real_df['y'] = real_df.apply(lambda x: x.y+2000 if x.extension>1 else x.y, axis=1)

In [4]:
#establish cuts from below line
#$5<=2.5 && $7*$7<=0.09 && $11<=2 && $20>=5 && $24==0 && $33>=5 && $37==0'' > ${TARG}.phot2")

real_cut = real_df[(real_df['chi'] < 2.5) & (real_df['sharp']*real_df['sharp'] <= 0.09) & (real_df['type'] <= 2) 
        & (real_df['snr_V'] >= 5) & (real_df['flag_V'] == 0) & (real_df['snr_I'] >= 5) 
        & (real_df['flag_I'] == 0)]

real_cut[:5] #print for a quick look

,extension,chip,x,y,chi,snr,sharp,round,maj_ax,crowd,...,nrm_ct_rt_err_I,inst_vega_I,mag_I,Ierr,chi_I,snr_I,sharp_I,round_I,crowd_I,flag_I
92,1,1,4014.41,1389.62,1.81,208.6,-0.005,0.016,0,0.000,...,1.540000e-11,21.461,21.451,0.006,1.62,168.9,0.001,0.040,0.000,0
118,1,1,347.72,1782.93,2.16,178.6,0.010,0.017,50,0.005,...,1.320000e-11,22.005,21.997,0.009,2.11,119.0,0.005,0.019,0.007,0
129,1,1,334.71,1687.48,1.91,179.5,0.001,0.050,110,0.000,...,1.330000e-11,21.830,21.820,0.008,1.94,139.1,0.006,0.028,0.000,0
658,1,1,3686.35,153.78,2.00,89.4,-0.012,0.120,110,0.011,...,8.840000e-12,22.884,22.872,0.014,2.37,79.5,-0.021,0.045,0.005,0
912,1,1,3633.93,805.82,1.34,79.4,-0.006,0.023,160,0.001,...,7.590000e-12,23.361,23.352,0.018,1.28,59.7,-0.026,0.048,0.000,0


In [5]:
#print out real_cut to space separated file
real_cut_file = real_cut.to_csv('%s.phot2' % (prefix), sep=' ', header = True, index = False)

In [6]:
#fake photometry next
fake_file='%s.phot.fake' % prefix #select file 

In [7]:
fake_df = pd.read_csv(fake_file, delim_whitespace=True, header=None) #create data frame 

# calculate number of columns
numberOfColumns = len(fake_df.columns)
numberOfExtraColumns = numberOfColumns - 71
numberOfImagesPerFilter = int(1+(numberOfExtraColumns/30))
numberOfRelevantColumns = (numberOfImagesPerFilter-1)*4

fake_df = fake_df.iloc[:,:(45+(numberOfRelevantColumns))] #select till relevant end of data frame (quality flag of I)

In [8]:
base_columns=['extensionIn','chipIn','xIn','yIn','counts_V1','mag_V1','counts_I1', 'mag_I1','extensionOut','chipOut','xOut','yOut',
           'chi','snr','sharp','round','maj_ax','crowd','type','cts_V','sky_v','nrm_ct_rt_V','nrm_ct_rt_err_V',
           'inst_vega_V','mag_V','Verr','chi_V','snr_V','sharp_V','round_V','crowd_V','flag_V','cts_I','sky_I',
           'nrm_ct_rt_I','nrm_ct_rt_err_I', 'inst_vega_I','mag_I','Ierr','chi_I','snr_I','sharp_I','round_I',
           'crowd_I','flag_I'] #base columns

In [9]:
#loop to find correct number of additional photometry columns, and insert them
j = 1
k = 1
l = 1

while (j < numberOfImagesPerFilter):
    m = str(j+1)
    base_columns.insert(k+5,'counts_V%s' % m) 
    base_columns.insert(k+6,'mag_V%s' % m)    
    base_columns.insert(l+9,'counts_I%s' % m) 
    base_columns.insert(l+10,'mag_I%s' % m)    
    j += 1
    k += 2
    l += 4
if j == numberOfImagesPerFilter:
    fake_df.columns=base_columns 
    
# fake_df.columns #print for a quick look

In [10]:
# correct Y position if the star is in the upper CCD
fake_df['yIn'] = fake_df.apply(lambda x: x.yIn+2000 if x.extensionIn>1 else x.yIn, axis=1) 
# fake_df.iloc[:5,:16] #print some relevant columns to see if correct

In [11]:
#make some selection cuts
fake_df['selected']=np.where(
    (fake_df['chi'] < 2.5) & (fake_df['sharp']*fake_df['sharp'] <= 0.09) & (fake_df['type'] <= 2) 
    & (fake_df['snr_V'] >= 5) & (fake_df['flag_V'] == 0) & (fake_df['snr_I'] >= 5) 
    & (fake_df['flag_I'] == 0),True, False)

# fake_df['selected'][:10] #print some relevant columns to see if correct

In [12]:
#automation depending on number of filters we have

#initalize columns to store denominator for magnitude conversion
fake_df['vSumCounts'] = 0
fake_df['iSumCounts'] = 0

#find the denominator (sum of counts)
for i in range(numberOfImagesPerFilter):
    vColumnName = 'counts_V%d' %(i+1)
    iColumnName = 'counts_I%d' %(i+1)
    fake_df['vSumCounts'] =  fake_df['vSumCounts'] + fake_df[vColumnName]
    fake_df['iSumCounts'] =  fake_df['iSumCounts'] + fake_df[iColumnName]

In [13]:
#convert to magnitudes
fake_df['dV'] = fake_df.apply(lambda x: np.round(-2.5*np.log10(x.cts_V/(x.vSumCounts)),3) 
                          if x.selected is True else 99.999, axis=1)
fake_df['dI'] = fake_df.apply(lambda x: np.round(-2.5*np.log10(x.cts_I/(x.iSumCounts)),3) 
                          if x.selected is True else 99.999, axis=1)

In [14]:
# #compare to old technique to make sure (this is assuming 2 input files in each filter)
# fake_df['dVOG'] = fake_df.apply(lambda x: np.round(-2.5*np.log10(x.cts_V/(x.counts_V1+x.counts_V2)),3) 
#                           if x.selected is True else 99.999, axis=1)
# fake_df['dIOG'] = fake_df.apply(lambda x: np.round(-2.5*np.log10(x.cts_I/(x.counts_I1+x.counts_I2)),3) 
#                           if x.selected is True else 99.999, axis=1)

# fake_df['dVdiffTest'] = fake_df['dV'] - fake_df['dVOG']
# fake_df['dIdiffTest'] = fake_df['dI'] - fake_df['dIOG']

In [15]:
# fake_df[fake_df['dVdiffTest'] != 0] #show any place there is a mismatch

,extensionIn,chipIn,xIn,yIn,counts_V1,mag_V1,counts_V2,mag_V2,counts_I1,mag_I1,...,flag_I,selected,vSumCounts,iSumCounts,dV,dI,dVOG,dIOG,dVdiffTest,dIdiffTest


In [16]:
# fake_df[fake_df['dIdiffTest'] != 0] #show any place there is a mismatch

,extensionIn,chipIn,xIn,yIn,counts_V1,mag_V1,counts_V2,mag_V2,counts_I1,mag_I1,...,flag_I,selected,vSumCounts,iSumCounts,dV,dI,dVOG,dIOG,dVdiffTest,dIdiffTest


In [17]:
#copy columns and create file
fake2_df = fake_df[['mag_V1','mag_I1','inst_vega_V','inst_vega_I','dV','dI']].copy()
fake2_df[:5] #print to have a quick look
fake_cut_file = fake2_df.to_csv('%s.phot.fake2' % (prefix), sep=' ', header = True, index = False)